# Using a CNN to classify DSF data

In [24]:
# import all the modules/classes/functions
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, MaxPool1D, Softmax, Dropout, Input, Flatten, BatchNormalization, Reshape
from keras.layers import LeakyReLU
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import to_categorical
import numpy as np
import pandas as pd

In [31]:
# import data and labels from stored archive and encode labels
def import_data(filename, num_classes=4):
    """imports the data/labels from a npz file and stores it into two np arrays,
    encodes the labels using the one-hot-encoding with the num_classes param"""
    with np.load(filename) as npzfile:
        data = npzfile["data"]
        labels = npzfile["labels"]
    # convert labels to one-hot-encoding
    labels_enc = to_categorical(labels, num_classes=num_classes)
    return data, labels_enc

#load augmented non-scaled data
data_ns_aug, labels_ns_aug = import_data("data_targets_aug.npz")

# shuffle the data/labels arrays consistently (only for augmented data)
data_ns_aug, labels_ns_aug = shuffle(data_ns_aug, labels_ns_aug, random_state=123)

# reshape data into a rank 3 tensor: (n_samples, 100, 1). Required by 1D conv layer
data_ns_aug = data_ns_aug.reshape(data_ns_aug.shape[0], data_ns_aug.shape[1], 1)
#display(data)
#display(labels)
print(data_ns_aug.shape, labels_ns_aug.shape)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

(3700, 100, 1) (3700, 4)


# Ideas and things to try

## Types of NNs:

1. CNNs
2. RNNs (LSTM)

## CNN architectures/params to try:

- 2x 1D convolutions (#filters=16, filter_size=3, activation=ReLU)
- Batch normalization (?) (maybe I add it after each conv)
- Max pooling after each conv step (size=2)
- Flatten tensor prior to 1st FC layer
- 1 FC hidden layer (32 units to start with, activation=ReLU)
- 1 output layer (4 units, activation=Softmax)
- If overfitting, add dropout layers! (p=0.5)

### backprop params:

- loss func: categorical cross-entropy
- optm. algorithm: adam (or sgd)

In [3]:
# construct a test network

class NeuralTester(object):
    """class for testing neural net architectures"""
    def __init__(self, base_build_fn, data, labels, epochs=50, batch_size=32, keras_api='sequential'):
        """base_build_fn - a function object that builds the basic network (returns a keras model)
        data - rank 2 tensor of shape (n_samples, 100)
        labels - binarized labels, rank 2 tensor of shape (n_samples, 4)"""
        self.base_build_fn = base_build_fn
        self.data = data
        self.labels = labels
        self.epochs = epochs
        self.batch_size = batch_size
        self.keras_api = keras_api
        
    def fit_predict(self, build_fn, scikit_wrapper, target_name):
        """Create scikit-estimator from a keras build function, fit to training, predict on test"""
                
        # Information for user
        print("\nFitting the scikit estimator {}\n{}".format(scikit_wrapper.__name__, '='*40))
        build_fn().summary() # create a keras model (build the graph)
        
        # Create the sklearn estimator (either KerasClassifier or KerasRegressor)
        estimator = scikit_wrapper(
            build_fn=build_fn, 
            epochs=self.epochs, 
            batch_size=self.batch_size, 
            verbose=1
        )
        
        # split data to train/validation sets
        data_train, data_test, labels_train, labels_test = train_test_split(self.data, self.labels,
                                                                           test_size=0.3,
                                                                           random_state=0,
                                                                           stratify=self.labels)
        
        # Fit the estimator using the training set
        estimator.fit(data_train, labels_train, 
            validation_data=(data_test, labels_test), 
            callbacks=[
                ReduceLROnPlateau(patience=3, verbose=1, factor=0.1),
                EarlyStopping(patience=5, verbose=1)
            ]
        )
        
        # Return predictions on test set
        return estimator.predict(data_test)
    
    def create_build_fn(self, loss, activation, optimizer='adam', metrics=[]):
        """Creates a final build function, depending on the task at hand and the model API used"""
        if self.keras_api == 'sequential':
            def build_fn():
                model = self.base_build_fn()
                model.add(Dense(4, activation=activation)) # 4 classes
                model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
                return model
            return build_fn
        else:
            def build_fn():
                inputs, outputs = self.base_build_fn()
                outputs = Dense(1, activation=activation)(outputs)
                model = Model(inputs=inputs, outputs=outputs)
                model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
                return model
            return build_fn
        
    def run_regression(self):
        """Use the build function to attempt regression task"""
        return self.fit_predict(
            self.create_build_fn('mean_squared_error', 'linear'), 
            KerasRegressor, 
            'y_regr'
        )
        
    def run_classification(self):
        """Use the build function to attempt classification task"""
        return self.fit_predict(
            self.create_build_fn('categorical_crossentropy', 'softmax', metrics=['accuracy']), 
            KerasClassifier, 
            'y_cls'
        )



## Build a simple FC model and test its performance

In [11]:
def create_model_FC():
    """creates a simple MLP"""
    model = Sequential()
    model.add(Flatten(input_shape=(100,1)))
    model.add(Dense(16, activation="relu"))
    return model

tester_fc = NeuralTester(create_model_FC, data_ns_aug, labels_ns_aug, epochs=100, batch_size=100)
clss_fc = tester_fc.run_classification()


Fitting the scikit estimator KerasClassifier
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_13 (Flatten)         (None, 100)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 16)                1616      
_________________________________________________________________
dense_26 (Dense)             (None, 4)                 68        
Total params: 1,684
Trainable params: 1,684
Non-trainable params: 0
_________________________________________________________________
Train on 2590 samples, validate on 1110 samples
Epoch 1/100
2590/2590 [==============================] - 1s 238us/step - loss: 1.4613 - acc: 0.3220 - val_loss: 1.3131 - val_acc: 0.3883
Epoch 2/100
2590/2590 [==============================] - 0s 15us/step - loss: 1.3665 - acc: 0.3969 - val_loss: 1.3043 - val_acc: 0.4045
Epoch 3/100
2590/2590 [==============

## Build a 1D conv model and test its performance

In [32]:
def create_model_1Dconv():
    """creates a start model to be completed by the NeuralTester class"""
    model = Sequential()
    
    model.add(Conv1D(4, 7, strides=2, input_shape=(100, 1), activation="relu"))
    #model.add(BatchNormalization())
    #model.add(LeakyReLU(alpha=0.3))
    #model.add(Conv1D(16, 3, activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    
    model.add(Conv1D(4, 5, strides=1, activation="relu"))
    #model.add(BatchNormalization())
    #model.add(LeakyReLU(alpha=0.3))
    #model.add(Conv1D(16, 3, activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    
    #model.add(Conv1D(16, 5))
    #model.add(BatchNormalization())
    #model.add(LeakyReLU(alpha=0.3))
    #model.add(Conv1D(16, 3, activation='relu'))
    #model.add(MaxPool1D(pool_size=2))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    return model

# Instantiate our testing class and fit regression & classification
tester = NeuralTester(create_model_1Dconv, data_ns_aug, labels_ns_aug, epochs=150, batch_size=64)
clss = tester.run_classification()

# Visual inspection of results
#visual_eval(data['y_regr']['test'], pred, cls)


Fitting the scikit estimator KerasClassifier
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_57 (Conv1D)           (None, 47, 4)             32        
_________________________________________________________________
max_pooling1d_56 (MaxPooling (None, 23, 4)             0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 19, 4)             84        
_________________________________________________________________
max_pooling1d_57 (MaxPooling (None, 9, 4)              0         
_________________________________________________________________
flatten_31 (Flatten)         (None, 36)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 64)                2368      
_________________________________________________________________
dense_62 (Dense)             (

Epoch 51/150
2590/2590 [==============================] - 0s 52us/step - loss: 0.8855 - acc: 0.6382 - val_loss: 0.9142 - val_acc: 0.6279
Epoch 52/150
2590/2590 [==============================] - 0s 53us/step - loss: 0.8792 - acc: 0.6386 - val_loss: 0.9209 - val_acc: 0.6180
Epoch 53/150
2590/2590 [==============================] - 0s 54us/step - loss: 0.8786 - acc: 0.6409 - val_loss: 0.9141 - val_acc: 0.6495
Epoch 54/150
2590/2590 [==============================] - 0s 53us/step - loss: 0.8745 - acc: 0.6440 - val_loss: 0.9067 - val_acc: 0.6252
Epoch 55/150
2590/2590 [==============================] - 0s 54us/step - loss: 0.8744 - acc: 0.6398 - val_loss: 0.9088 - val_acc: 0.6045
Epoch 56/150
2590/2590 [==============================] - 0s 54us/step - loss: 0.8707 - acc: 0.6452 - val_loss: 0.9055 - val_acc: 0.6261
Epoch 57/150
2590/2590 [==============================] - 0s 55us/step - loss: 0.8620 - acc: 0.6486 - val_loss: 0.9192 - val_acc: 0.6045
Epoch 58/150
2590/2590 [=================

# Tune the hyperparameters of the neural net

These parameters could be:
-  filter size for conv nets (int)
-  num. of filters for each conv layer (int)
-  num. of units in the FC layer (int)
-  dropout rate (float)
-  etc.

One could use either a grid search or a Baysean optimization process to find an optimal set of hyperparameters. In each case a new net is constructed and evaluated using CV on the validation set. The best net is then evaluated against a final test set which was set aside in the beginning